In [54]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "02" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 42
repetitionName = "udrlBC2special"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

5
0.5555555555555556


In [55]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 198)
(10, 198)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_16776\1128782281.py:188: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,18,19,20,21,22,23,24,25,26,48,49,50,51,52,53,54,55,56,75,76,77,78,79,80,81,82,83,99,100,101,102,103,104,105,106,107,120,121,122,123,124,125,126,127,128,138,139,140,141,142,143,144,145,146,153,154,155,156,157,158,159,160,161,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.775,0.775,0.8,0.575,0.625,0.625,0.75,0.75,0.725,0.725,0.725,0.75,0.6,0.625,0.65,0.725,0.75,0.725,0.75,0.725,0.8,0.575,0.625,0.625,0.75,0.75,0.725,0.775,0.725,0.775,0.625,0.575,0.625,0.725,0.75,0.725,0.75,0.7,0.7,0.65,0.65,0.7,0.75,0.725,0.775,0.775,0.775,0.8,0.575,0.625,0.625,0.75,0.75,0.725,0.725,0.75,0.775,0.575,0.625,0.625,0.75,0.75,0.725,0.725,0.675,0.775,0.65,0.725,0.675,0.775,0.75,0.8,0.725,0.7,0.75,0.75,0.725,0.725,0.6,0.625,0.65,0.625,0.6,0.65,0.75,0.75,0.725,0.75,0.7,0.75
2,0.75,0.75,0.791667,0.770833,0.729167,0.75,0.645833,0.6875,0.645833,0.75,0.75,0.791667,0.770833,0.729167,0.770833,0.6875,0.708333,0.625,0.75,0.75,0.791667,0.770833,0.729167,0.770833,0.666667,0.729167,0.645833,0.729167,0.75,0.770833,0.770833,0.75,0.729167,0.666667,0.708333,0.625,0.770833,0.8125,0.791667,0.791667,0.75,0.791667,0.729167,0.708333,0.729167,0.75,0.75,0.791667,0.770833,0.729167,0.770833,0.645833,0.6875,0.645833,0.75,0.75,0.791667,0.75,0.729167,0.770833,0.666667,0.729167,0.645833,0.729167,0.729167,0.770833,0.708333,0.6875,0.770833,0.75,0.75,0.791667,0.729167,0.770833,0.791667,0.708333,0.666667,0.729167,0.75,0.729167,0.791667,0.770833,0.75,0.75,0.6875,0.708333,0.625,0.770833,0.708333,0.729167
3,0.777778,0.75,0.694444,0.722222,0.777778,0.722222,0.611111,0.583333,0.666667,0.75,0.722222,0.722222,0.722222,0.75,0.75,0.611111,0.583333,0.694444,0.777778,0.75,0.722222,0.722222,0.805556,0.722222,0.611111,0.583333,0.694444,0.722222,0.75,0.694444,0.75,0.722222,0.75,0.611111,0.638889,0.694444,0.694444,0.694444,0.694444,0.777778,0.722222,0.777778,0.583333,0.555556,0.638889,0.777778,0.75,0.694444,0.722222,0.777778,0.722222,0.611111,0.583333,0.666667,0.75,0.722222,0.694444,0.722222,0.75,0.722222,0.611111,0.555556,0.666667,0.805556,0.75,0.75,0.694444,0.722222,0.722222,0.777778,0.75,0.722222,0.75,0.666667,0.722222,0.666667,0.694444,0.694444,0.722222,0.75,0.722222,0.75,0.666667,0.75,0.611111,0.583333,0.694444,0.527778,None,0.611111
4,0.791667,0.8125,0.770833,0.75,0.75,0.708333,0.666667,0.645833,0.604167,0.791667,0.8125,0.770833,0.75,0.770833,0.729167,0.729167,0.666667,0.583333,0.770833,0.8125,0.791667,0.75,0.75,0.729167,0.666667,0.666667,0.5625,0.75,0.8125,0.770833,0.75,0.729167,0.708333,0.6875,0.666667,0.583333,0.75,0.75,0.75,0.791667,0.770833,0.75,0.75,0.625,0.604167,0.791667,0.8125,0.770833,0.75,0.75,0.708333,0.666667,0.645833,0.604167,0.791667,0.791667,0.770833,0.75,0.770833,0.729167,0.6875,0.645833,0.583333,0.75,0.770833,0.729167,0.729167,0.6875,0.75,0.791667,0.8125,0.770833,0.75,0.729167,0.708333,0.708333,0.708333,0.6875,0.75,0.75,0.75,0.8125,0.791667,0.729167,0.6875,0.645833,0.583333,0.708333,0.645833,0.625
5,0.791667,0.708333,0.708333,0.8125,0.75,0.6875,0.604167,0.708333,0.6875,0.770833,0.708333,0.6875,0.833333,0.729167,0.708333,0.6875,0.729167,0.708333,0.770833,0.708333,0.6875,0.791667,0.75,0.729167,0.6875,0.6875,0.708333,0.8125,0.708333,0.729167,0.791667,0.708333,0.708333,0.604167,0.729167,0.6875,0.833333,0.75,0.666667,0.833333,0.729167,0.729167,0.625,0.6875,0.708333,0.791667,0.708333,0.708333,0.8125,0.75,0.708333,0.604167,0.708333,0.6875,0.791667,0.708333,0.6875,0.833333,0.708333,0.729167,0.645833,0.708333,0.6875,0.8125,0.708333,0.729167,0.770833,0.75,0.770833,0.8125,0.70833

In [56]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.83 : subject 5, feature:welchDataBC--fftData, kernellinear, C:0.5
Accuracy 0.83 : subject 5, feature:welchDataBC--dataHRCV, kernellinear, C:0.5
Accuracy 0.83 : subject 5, feature:welchDataBC--dataFFTCV, kernellinear, C:0.5
Accuracy 0.83 : subject 5, feature:fftDatacn6--welchDataBC, kernellinear, C:0.5
Accuracy 0.83 : subject 5, feature:fftDataBC--dataFFTCV, kernellinear, C:0.5
Accuracy 0.81 : subject 5, feature:-welchDataBC, kernellinear, C:0.5
Accuracy 0.81 : subject 4, feature:fftDatacn6--fftDataBC, kernelrbf, C:2.5
Accuracy 0.81 : subject 4, feature:fftDataBC--welchData, kernelrbf, C:2.5
Accuracy 0.81 : subject 4, feature:fftDataBC--fftData, kernelrbf, C:2.5
Accuracy 0.81 : subject 4, feature:fftDataBC--dataWCV, kernelrbf, C:2.5


In [57]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.83 : subject 5, feature:welchDataBC--fftData, kernellinear, C:0.5
Accuracy 0.83 : subject 5, feature:fftDataBC--dataFFTCV, kernellinear, C:0.5

Accuracy 0.81 : subject 4, feature:fftDataBC--fftData, kernelrbf, C:2.5
Accuracy 0.81 : subject 4, feature:-fftDataBC, kernelrbf, C:2.5

Accuracy 0.81 : subject 2, feature:fftDataBC--dataFFTCV, kernelrbf, C:2.5
Accuracy 0.79 : subject 2, feature:fftDataBC--fftData, kernelsigmoid, C:2.5

Accuracy 0.81 : subject 3, feature:welchDataBC--welchData, kernelrbf, C:2.5
Accuracy 0.81 : subject 3, feature:welchDataBC--fftDataBC, kernellinear, C:0.5

Accuracy 0.8 : subject 1, feature:fftDataBC--welchData, kernelsigmoid, C:2.5
Accuracy 0.8 : subject 1, feature:-fftDataBC, kernelsigmoid, C:2.5


In [58]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.70555556 0.73402778 0.6994382  0.72430556 0.72615741
 0.         0.         0.         0.        ]


,18,19,20,21,22,23,24,25,26,48,49,50,51,52,53,54,55,56,75,76,77,78,79,80,81,82,83,99,100,101,102,103,104,105,106,107,120,121,122,123,124,125,126,127,128,138,139,140,141,142,143,144,145,146,153,154,155,156,157,158,159,160,161,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194
0,0.777222,0.759167,0.753056,0.726111,0.726389,0.698611,0.655556,0.675,0.665833,0.7575,0.743611,0.744444,0.735278,0.720833,0.721667,0.688056,0.6875,0.667222,0.763889,0.749167,0.758611,0.721944,0.731944,0.715278,0.676389,0.683333,0.667222,0.757778,0.749167,0.748056,0.7375,0.696944,0.704167,0.658889,0.698611,0.663056,0.759722,0.741389,0.720556,0.768889,0.724444,0.749722,0.6875,0.660278,0.691111,0.777222,0.759167,0.753056,0.726111,0.726389,0.706944,0.655556,0.675,0.665833,0.761667,0.744444,0.743889,0.726111,0.716667,0.715278,0.672222,0.677778,0.661667,0.764444,0.726667,0.750833,0.710556,0.714444,0.737778,0.781389,0.754167,0.754444,0.749167,0.715,0.744444,0.725,0.704722,0.717222,0.731111,0.716667,0.724444,0.75,0.7075,0.7175,0.684722,0.6875,0.667222,0.668056,0.659375,0.684722


Max average accuracy 0.7813888888888888
174
5.0
dataWCV--welchData
linear
C: 0.5


In [59]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-fftData
1
-welchData
2
-dataHR
3
-dataFFTCV
4
-dataWCV
5
-dataHRCV
6
-fftDataBC
7
-welchDataBC
8
-dataHRBC
9
-fftDatacn6
10
-dataFFTCVcn6
11
welchData--fftData
12
dataHR--fftData
13
dataFFTCV--fftData
14
dataWCV--fftData
15
dataHRCV--fftData
16
fftDataBC--fftData
17
welchDataBC--fftData
18
dataHRBC--fftData
19
fftDatacn6--fftData
20
dataFFTCVcn6--fftData
21
dataHR--welchData
22
dataFFTCV--welchData
23
dataWCV--welchData
24
dataHRCV--welchData
25
fftDataBC--welchData
26
welchDataBC--welchData
27
dataHRBC--welchData
28
fftDatacn6--welchData
29
dataFFTCVcn6--welchData
30
dataFFTCV--dataHR
31
dataWCV--dataHR
32
dataHRCV--dataHR
33
fftDataBC--dataHR
34
welchDataBC--dataHR
35
dataHRBC--dataHR
36
fftDatacn6--dataHR
37
dataFFTCVcn6--dataHR
38
dataWCV--dataFFTCV
39
dataHRCV--dataFFTCV
40
fftDataBC--dataFFTCV
41
welchDataBC--dataFFTCV
42
dataHRBC--dataFFTCV
43
fftDatacn6--dataFFTCV
44
dataFFTCVcn6--dataFFTCV
45
dataHRCV--dataWCV
46
fftDataBC--dataWCV
47
welchDataBC--dataWCV
48
dataHRBC--dataWCV

In [60]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,18,19,20,21,22,23,24,25,26,48,49,50,51,52,53,54,55,56,75,76,77,78,79,80,81,82,83,99,100,101,102,103,104,105,106,107,120,121,122,123,124,125,126,127,128,138,139,140,141,142,143,144,145,146,153,154,155,156,157,158,159,160,161,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194
0,3.886111,3.795833,3.765278,3.630556,3.631944,3.493056,3.277778,3.375,3.329167,3.7875,3.718056,3.722222,3.676389,3.604167,3.608333,3.440278,3.4375,3.336111,3.819444,3.745833,3.793056,3.609722,3.659722,3.576389,3.381944,3.416667,3.336111,3.788889,3.745833,3.740278,3.6875,3.484722,3.520833,3.294444,3.493056,3.315278,3.798611,3.706944,3.602778,3.844444,3.622222,3.748611,3.4375,3.301389,3.455556,3.886111,3.795833,3.765278,3.630556,3.631944,3.534722,3.277778,3.375,3.329167,3.808333,3.722222,3.719444,3.630556,3.583333,3.576389,3.361111,3.388889,3.308333,3.822222,3.633333,3.754167,3.552778,3.572222,3.688889,3.906944,3.770833,3.772222,3.745833,3.575,3.722222,3.625,3.523611,3.586111,3.655556,3.583333,3.622222,3.75,3.5375,3.5875,3.423611,3.4375,3.336111,3.340278,2.6375,3.423611


174
5.0
Max total significant accuracy then averaged 0.7813888888888888 
dataWCV--welchData
linear
C: 0.5 


In [61]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
